<h1> Exercice </h1>

Le dataset "Titres Netflix" est une compilation exhaustive de films et de séries télévisées disponibles sur Netflix, couvrant divers aspects tels que le type de titre, le réalisateur, les acteurs, le pays de production, l'année de sortie, la classification, la durée, les genres (répertoriés dans) et une brève description. Ce dataset est essentiel pour analyser les tendances du contenu Netflix, comprendre la popularité des genres et examiner la distribution du contenu à travers différentes régions et périodes.

Les colonnes : 
- show_id : Un identifiant unique pour chaque titre.
- type : La catégorie du titre, qui peut être 'Film' ou 'Série télévisée'.
- title : Le nom du film ou de la série télévisée.
- director : Le(s) réalisateur(s) du film ou de la série télévisée. (Contient des valeurs nulles pour certaines entrées, en particulier les séries télévisées où cette information peut ne pas être applicable.)
- cast : La liste des acteurs principaux du titre. (Certaines entrées peuvent ne pas avoir cette information.)
- country : Le pays ou les pays où le film ou la série télévisée a été produit.
- date_added : La date à laquelle le titre a été ajouté à Netflix.
- release_year : L'année de sortie originale du film ou de la série télévisée.
- rating : La classification par âge du titre.
- duration : La durée du titre, en minutes pour les films et en saisons pour les séries télévisées.
- listed_in : Les genres auxquels appartient le titre.
- description : Un bref résumé du titre.

<h1> Enoncé </h1>

<h2> data cleaning & data modeling </h2>

<b> 1) Importation du fichier </b>

Tentez d'importer le fichier netflix_titles.csv. Si vous rencontrez une erreur, celle-ci est probablement liée à un problème d'encodage. Trouvez l'encodage adéquat pour lire le fichier correctement.

<b> 2) Création d'une copie du DataFrame </b>

Créez une copie de votre DataFrame dans une nouvelle variable afin de conserver les données originales accessibles.

<b> 3) Suppression des colonnes inutiles </b>

Vérifiez que les colonnes nommées "Unnamed:..." en fin de DataFrame ne contiennent aucune donnée, puis supprimez-les.

<b> 4) Définition d'un nouvel index </b>

Assurez-vous que la colonne "Show_id" ne possède ni valeur nulle ni doublon. Si c'est le cas, définissez show_id comme index du DataFrame.

<b> 5) Correction du type de colonne </b>
Vérifiez le type de la colonne "Date_added" et corrigez-le avec le type adéquat si nécessaire.

<b> 6) Gestion de la colonne "Duration" </b>

- Confirmez que la colonne "Type" contient uniquement les valeurs 'Movie' et 'TV Show'.
- Examinez la nomenclature des valeurs de la colonne "Duration".
- Via un groupby, vérifiez que les durées sont bien associées à une durée en minutes pour les films et à un nombre de saisons pour les séries.
- Créez une nouvelle colonne 'duration (movies)' pour isoler le nombre de minutes pour les films. Assurez-vous que cette  colonne soit de type "float".
- Créez une colonne 'seasons (TV Show)' pour isoler le nombre de saisons pour les séries. Assurez-vous que cette colonne soit de type "float".
- Supprimez finalement la colonne 'Duration' du dataset.

<b> 7) Création de DataFrames annexes pour les valeurs séparées par des virgules </b>

Certaines colonnes comme country, cast et listed_in contiennent pour certains titres des séries de valeurs, sépérées par des virgules. Nous souhaitons créer des tableaux annexes permettant d'avoir chaque valeur de manière distincte.

Pour la colonne "Country" :
- Créez une colonne 'countries' en transformant les valeurs de la colonne 'country' en listes (via str.split).
- Créez un DataFrame 'countries_exploded' qui génère une ligne pour chaque pays via la méthode .explode().
- Isolez uniquement la colonne 'countries' dans le DataFrame 'countries_exploded' (non vue en cours, à vous de comprendre son fonctionnement : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html)
- Répétez ce processus pour les colonnes 'categories' (basée sur 'listed_in') et 'cast', créant respectivement les - DataFrames 'categories_exploded' et 'cast_list'.

<b> 8) Suppression des colonnes transformées </b>

Supprimez les trois colonnes transformées du DataFrame original.

<b> 9) Création de colonnes temporelles </b>

À partir de 'Date Added', créez des colonnes pour l'année d'ajout, le mois d'ajout et le jour de la semaine d'ajout.
Analyse des Données

<h2> Analyse des données </h2>

Tentez maintenant de répondre aux questions suivantes : 

1) Combien de "shows" sont présents dans ce dataset ?
2) Quelle est la répartition entre les types 'Movie' et 'TV Show' ?
3) Quelle est la répartition des ajouts en fonction de l'année ?
4) Quel est le top 5 des catégories de shows les plus ajoutées ?
5) Quel est le top 5 des comédiens les plus plébiscités aux États-Unis ?
6) Quelle est la répartition des ajouts en fonction du jour de la semaine ?
7) Dans quel pays sont produits le plus de documentaires ?
8) En moyenne, combien de saisons ont les séries ?
9) Quelle est la distribution des films en fonction de leur durée (quartiles) ?
10) Combien de shows ont pour thématique la drogue (présence du mot "drug" dans la description) ?

<h1> Data Cleaning et data modeling</h1>

<h1> Analyse des données </h1>

In [21]:
#1
import pandas as pd
import numpy as np
from pathlib import Path

# Chemin vers ton fichier
DATA_PATH = Path("netflix_titles.csv")

# 1) Importation du fichier avec gestion d'encodage
encodings_to_try = ["utf-8", "latin1", "iso-8859-1"]

for enc in encodings_to_try:
    try:
        df = pd.read_csv(DATA_PATH, encoding=enc)
        print(f"Fichier chargé avec succès avec l'encodage : {enc}")
        break
    except UnicodeDecodeError:
        print(f"Échec avec l'encodage : {enc}")
else:
    raise ValueError("Impossible de lire le fichier, encodage inconnu")


Échec avec l'encodage : utf-8
Fichier chargé avec succès avec l'encodage : latin1


In [22]:
#2
netflix = df.copy()


In [23]:
#3
# Colonnes qui commencent par "Unnamed"
cols_unnamed = [col for col in netflix.columns if col.startswith("Unnamed")]

print("Colonnes Unnamed détectées :", cols_unnamed)
print(netflix[cols_unnamed].isna().sum())  # vérifier qu'elles sont vides

netflix = netflix.drop(columns=cols_unnamed)


Colonnes Unnamed détectées : ['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25']
Unnamed: 12    8809
Unnamed: 13    8809
Unnamed: 14    8809
Unnamed: 15    8809
Unnamed: 16    8809
Unnamed: 17    8809
Unnamed: 18    8809
Unnamed: 19    8809
Unnamed: 20    8809
Unnamed: 21    8809
Unnamed: 22    8809
Unnamed: 23    8809
Unnamed: 24    8809
Unnamed: 25    8809
dtype: int64


In [24]:
#4

# Vérifier qu'il n'y a ni NaN ni doublons
print(netflix["show_id"].isna().sum())
print(netflix["show_id"].duplicated().sum())

# Si tout est OK :
netflix = netflix.set_index("show_id")


0
0


In [25]:
#5

netflix["date_added"] = pd.to_datetime(netflix["date_added"], errors="coerce")
netflix["date_added"].head()


show_id
s1   2021-09-25
s2   2021-09-24
s3   2021-09-24
s4   2021-09-24
s5   2021-09-24
Name: date_added, dtype: datetime64[ns]

In [26]:
#6

# Vérifier que type contient bien Movie / TV Show
print(netflix["type"].value_counts())

# Aperçu de la colonne duration
print(netflix["duration"].head())

# Séparer la partie numérique et la partie texte
# ex : "90 min" -> "90", "min" ; "2 Seasons" -> "2", "Seasons"
duration_split = netflix["duration"].str.extract(r"(?P<value>\d+)\s*(?P<unit>\w+)", expand=True)

netflix["duration_value"] = pd.to_numeric(duration_split["value"], errors="coerce")
netflix["duration_unit"] = duration_split["unit"].str.lower()

# Vérifier correspondance type / unité
print(netflix.groupby(["type", "duration_unit"]).size())

# Créer les nouvelles colonnes
netflix["duration_movies"] = np.where(
    netflix["type"] == "Movie",
    netflix["duration_value"],
    np.nan
).astype(float)

netflix["seasons_tvshow"] = np.where(
    netflix["type"] == "TV Show",
    netflix["duration_value"],
    np.nan
).astype(float)

# Supprimer les colonnes temporaires
netflix = netflix.drop(columns=["duration", "duration_value", "duration_unit"])


type
Movie      6132
TV Show    2677
Name: count, dtype: int64
show_id
s1       90 min
s2    2 Seasons
s3     1 Season
s4     1 Season
s5    2 Seasons
Name: duration, dtype: object
type     duration_unit
Movie    min              6129
TV Show  season           1794
         seasons           883
dtype: int64


In [27]:
#7

# ----- countries -----
netflix["countries"] = netflix["country"].str.split(r"\s*,\s*")

countries_exploded = (
    netflix[["countries"]]
    .explode("countries")
    .rename(columns={"countries": "country"})
)

# ----- categories (à partir de listed_in) -----
netflix["categories"] = netflix["listed_in"].str.split(r"\s*,\s*")

categories_exploded = (
    netflix[["categories"]]
    .explode("categories")
)

# ----- cast -----
netflix["cast_list"] = netflix["cast"].str.split(r"\s*,\s*")

cast_exploded = (
    netflix[["cast_list"]]
    .explode("cast_list")
    .rename(columns={"cast_list": "actor"})
)


In [28]:
#8
netflix = netflix.drop(columns=["country", "listed_in", "cast"])


In [29]:
#9

netflix["year_added"] = netflix["date_added"].dt.year
netflix["month_added"] = netflix["date_added"].dt.month
netflix["weekday_added"] = netflix["date_added"].dt.day_name()  # en anglais
# Si tu veux en français :
# netflix["weekday_added"] = netflix["date_added"].dt.day_name(locale="fr_FR")


In [30]:
#1) Nombre de shows dans le dataset
nb_shows = len(netflix)
print("Nombre de shows :", nb_shows)

Nombre de shows : 8809


In [31]:
#2) Répartition Movie / TV Show

type_counts = netflix["type"].value_counts()
type_ratio = netflix["type"].value_counts(normalize=True) * 100

print("Répartition en nombre :")
print(type_counts)
print("\nRépartition en % :")
print(type_ratio.round(2))


Répartition en nombre :
type
Movie      6132
TV Show    2677
Name: count, dtype: int64

Répartition en % :
type
Movie      69.61
TV Show    30.39
Name: proportion, dtype: float64


In [32]:
#3) Répartition des ajouts par année
ajouts_par_annee = netflix["year_added"].value_counts().sort_index()
print(ajouts_par_annee)


year_added
2008.0       2
2009.0       2
2010.0       1
2011.0      13
2012.0       3
2013.0      10
2014.0      23
2015.0      73
2016.0     418
2017.0    1164
2018.0    1625
2019.0    1999
2020.0    1878
2021.0    1498
2024.0       2
Name: count, dtype: int64


In [33]:
#4) Top 5 des catégories les plus ajoutées

top5_categories = categories_exploded["categories"].value_counts().head(5)
print(top5_categories)


categories
International Movies      2752
Dramas                    2427
Comedies                  1674
International TV Shows    1351
Documentaries              869
Name: count, dtype: int64


In [34]:
#5) Top 5 des comédiens les plus présents aux États-Unis

# Index (show_id) des titres qui incluent "United States" dans leurs pays
us_titles_index = countries_exploded[
    countries_exploded["country"] == "United States"
].index

# On récupère les acteurs pour ces shows
us_actors = cast_exploded.loc[us_titles_index, "actor"].dropna()

top5_actors_us = us_actors.value_counts().head(5)
print(top5_actors_us)


actor
Samuel L. Jackson    22
Tara Strong          22
Fred Tatasciore      21
Adam Sandler         20
Nicolas Cage         19
Name: count, dtype: int64


In [35]:
#6) Répartition des ajouts par jour de la semaine
ajouts_par_jour = netflix["weekday_added"].value_counts()
print(ajouts_par_jour)


weekday_added
Friday       2478
Thursday     1387
Wednesday    1276
Tuesday      1182
Monday        845
Saturday      803
Sunday        740
Name: count, dtype: int64


In [36]:
#7) Pays qui produit le plus de documentaires

doc_index = categories_exploded[
    categories_exploded["categories"].str.contains("Documentaries", na=False)
].index

doc_countries = countries_exploded.loc[doc_index, "country"].dropna()

pays_plus_doc = doc_countries.value_counts().head(10)
print(pays_plus_doc)


country
United States     512
United Kingdom    128
France             44
Canada             42
India              27
Germany            21
Spain              21
Mexico             18
Italy              17
Australia          15
Name: count, dtype: int64


In [37]:
#8) Nombre moyen de saisons des séries

mean_seasons = netflix.loc[netflix["type"] == "TV Show", "seasons_tvshow"].mean()
print("Nombre moyen de saisons :", mean_seasons)


Nombre moyen de saisons : 1.7646619350018677


In [38]:
#9) Distribution des films par durée (quartiles)

films = netflix[netflix["type"] == "Movie"]
quartiles = films["duration_movies"].quantile([0.25, 0.5, 0.75])
print(quartiles)


0.25     87.0
0.50     98.0
0.75    114.0
Name: duration_movies, dtype: float64


In [39]:
#10) Nombre de shows avec “drug” dans la description

nb_drug = netflix["description"].str.contains("drug", case=False, na=False).sum()
print("Nombre de shows liés à la drogue :", nb_drug)


Nombre de shows liés à la drogue : 161
